In [ ]:
# This allows changes in chakra.py to be automatically re-imported
# (this is tricky with OOP though, to be used with care)
%load_ext autoreload
%autoreload 1
%aimport chakra

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# Get the default params
from oggm import cfg
cfg.initialize(logging_level='WORKFLOW')
cfg.PARAMS['cfl_number'] = 0.01  # less numerical instabilities
cfg.PARAMS['use_multiprocessing'] = False

In [ ]:
from oggm.core.massbalance import ScalarMassBalance
no_mb = ScalarMassBalance()

# BU bed

In [ ]:
bu_fl = chakra.bu_tidewater_bed()[0]

xc = bu_fl.dis_on_line * bu_fl.dx_meter / 1000
f, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.plot(xc, bu_fl.bed_h, color='k')
plt.hlines(0, *xc[[0, -1]], color='C0', linestyles=':')
plt.ylim(-350, 1000); plt.ylabel('Altitude [m]'); plt.xlabel('Distance along flowline [km]');

# K-calving param

Here I just want to check that the k-calving param is also applicable with the "parameterization" mechanism in Chakra which, after our discussion from just now, seems useless anyway:

In [ ]:
def k_calving(model, dt):
    """k-calving parameterization from Oorlemans 2008

    Parameters
    ----------
    model : the instance of the Chakra model at the moment it
        calls this function
    dt : the chosen timestep according to CFL
    """
    
    # Calving k needs to be defined at the top level by caller
    k = calving_k / cfg.SEC_IN_YEAR
    
    model.calving_rate_myr = 0.
    
    fl = model.fls[-1]

    # No need to do calving in these cases
    if not model.do_calving or not fl.has_ice():
        return

    # We do calving only if the last glacier bed pixel is below water
    # (this is to avoid calving elsewhere than at the front)
    if fl.bed_h[fl.thick > 0][-1] > model.water_level:
        return

    # We do calving only if there is some ice above wl
    last_above_wl = np.nonzero((fl.surface_h > model.water_level) &
                               (fl.thick > 0))[0][-1]
    if fl.bed_h[last_above_wl] > model.water_level:
        return

    # OK, so we're really calving
    section = fl.section

    # Calving law
    h = fl.thick[last_above_wl]
    d = h - (fl.surface_h[last_above_wl] - model.water_level)
    q_calving = k * d * h * fl.widths_m[last_above_wl]
    # Add to the bucket and the diagnostics
    fl.calving_bucket_m3 += q_calving * dt
    model.calving_m3_since_y0 += q_calving * dt
    model.calving_rate_myr = (q_calving / section[last_above_wl] *
                              cfg.SEC_IN_YEAR)

    # See if we have ice below sea-water to clean out first
    below_sl = (fl.surface_h < model.water_level) & (fl.thick > 0)
    to_remove = np.sum(section[below_sl]) * fl.dx_meter
    if 0 < to_remove < fl.calving_bucket_m3:
        # This is easy, we remove everything
        section[below_sl] = 0
        fl.calving_bucket_m3 -= to_remove
    elif to_remove > 0:
        # We can only remove part of if
        section[below_sl] = 0
        section[last_above_wl + 1] = (to_remove - fl.calving_bucket_m3) / fl.dx_meter
        fl.calving_bucket_m3 = 0

    # The rest of the bucket might calve an entire grid point
    vol_last = section[last_above_wl] * fl.dx_meter
    if fl.calving_bucket_m3 > vol_last:
        fl.calving_bucket_m3 -= vol_last
        section[last_above_wl] = 0

    # We update the glacier with our changes
    fl.section = section

### Equilibrium states 

In [ ]:
calving_k = 0.2 # default calving is huge

to_plot = None
keys = []
for flux_gate in [0.06, 0.10, 0.16]:
    
    model = chakra.ChakraModel(chakra.bu_tidewater_bed(), mb_model=no_mb, 
                               flux_gate=flux_gate,
                               calving_use_limiter=True,
                               do_calving=True,
                               apply_parameterization=k_calving,
                               )
    
    # long enough to reach approx. equilibrium 
    _, ds = model.run_until_and_store(6000)
    df_diag = model.get_diagnostics()
    
    if to_plot is None:
        to_plot = df_diag
    
    key = 'Flux gate={:.02f}. Calving rate: {:.0f} m yr-1'.format(flux_gate, model.calving_rate_myr)
    to_plot[key] = df_diag['surface_h']
    keys.append(key)
    
    # Plot of volume
    (ds.volume_m3 * 1e-9).plot(label=key);
    
plt.legend(); plt.ylabel('Volume [km$^{3}$]');
to_plot.index = xc

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12, 5))
to_plot[keys].plot(ax=ax);
to_plot.bed_h.plot(ax=ax, color='k')
plt.hlines(0, *xc[[0, -1]], color='C0', linestyles=':')
plt.ylim(-350, 1000); plt.ylabel('Altitude [m]'); plt.xlabel('Distance along flowline [km]');

### Periodic forcing

Repeat the BU paper figure:

In [ ]:
# Periodic forcing
years = np.arange(6001)
flux = 0.4 + 0.4 * np.sin(2 * np.pi * years / 5000)
def flux_gate(year):
    return flux[int(year)]

In [ ]:
calving_k = 1  # a bit more calving than before

model = chakra.ChakraModel(chakra.bu_tidewater_bed(), mb_model=no_mb,
                           glen_a=cfg.PARAMS['glen_a']*3, # make the glacier flow faster
                           flux_gate=flux_gate,  # default is 0
                           calving_use_limiter=True,
                           do_calving=True,
                           apply_parameterization=k_calving,
                      )
t0 = time.time()
_, ds = model.run_until_and_store(len(flux)-1)
print('Done! Time needed: {}s'.format(int(time.time()-t0)))

In [ ]:
# Prepare the data for plotting
df = (ds.volume_m3 * 1e-9).to_dataframe(name='Volume [km$^3$]')[['Volume [km$^3$]']]
df['Length [m]'] = (ds['length_m'] / 1000).to_series()
df['Calving rate [m y$^{-1}$]'] = ds['calving_rate_myr'].to_series()
df['Forcing'] = flux

# Thresholds
deep_val = 27
dfs = df.loc[(df['Length [m]'] >= deep_val) & (df.index < 5000)]
deep_t0, deep_t1 = dfs.index[0], dfs.index[-1]
dfs = df.loc[(df['Length [m]'] >= deep_val) & (df.index > 5000)]
deep_t2 = dfs.index[0]

bump_val = 37.5
dfs = df.loc[(df['Length [m]'] >= bump_val) & (df.index < 5000)]
bump_t0, bump_t1 = dfs.index[0], dfs.index[-1]

In [ ]:
# The plot
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(9, 9), sharex=True)
ts = df['Forcing']
ts.plot(ax=ax1, color='C0');
ax1.set_ylabel(ts.name)
ts = df['Length [m]']
ts.plot(ax=ax2, color='C1');
ax2.hlines(deep_val, deep_t0, deep_t1, color='black', linestyles=':')
ax2.hlines(deep_val, deep_t2, 6000, color='black', linestyles=':')
ax2.hlines(bump_val, bump_t0, bump_t1, color='grey', linestyles='--')
ax2.annotate('Deepening', (deep_t0, deep_val-5))
ax2.annotate('Bump', (bump_t0, bump_val-5))
ax2.set_ylabel(ts.name)
# The calving rate is a bit noisy because of the bucket trick - we smooth
ts = df['Calving rate [m y$^{-1}$]'].rolling(11, center=True).max()
ts.plot(ax=ax3, color='C3')
ax3.vlines([deep_t0, deep_t1, deep_t2], ts.min(), ts.max(), color='black', linestyles=':')
ax3.vlines([bump_t0, bump_t1], ts.min(), ts.max(), color='grey', linestyles='--');
ax3.set_ylabel(ts.name); ax3.set_xlabel('Years');

# Now, do the same with Sermeq? 

As discussed today, the only thing we have to agree on are the bed and the flux of ice on the left-hand side of the domain.

The bed is:

In [ ]:
# Equations in chakra.py
bu_fl = chakra.bu_tidewater_bed()[0]

# These two things are numpy arrays
xc = bu_fl.dis_on_line * bu_fl.dx_meter / 1000
bed_h = bu_fl.bed_h

f, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.plot(xc, bed_h, color='k')
plt.hlines(0, *xc[[0, -1]], color='C0', linestyles=':')
plt.ylim(-350, 1000); plt.ylabel('Altitude [m]'); plt.xlabel('Distance along flowline [km]');

The flux I used above (which is tuned so that it works nice with the chosen OGGM params, so you may adapt it so it works for you, and I'll try to get OGGM to play nicely):

In [ ]:
# for the equilibrium states: [0.06, 0.10, 0.16] m3 of ice per second
# to convert to m2 per second to remove the width dimension:
bu_fl.widths_m[0]

For the periodic case it is one order of magnitude larger - I deal with this higher flux with higher ice deformation and higher calving rates, leading to higher ice velocities, so that I can handle the flux:

In [ ]:
# Still units of m3 of ice per second
flux_gate(6000)
# Just for an order of magnitude, this is what it looks like in m yr-1 at the end of the simulation:
flux_gate(6000) / bu_fl.widths_m[0] / model.fls[0].thick[0] * cfg.SEC_IN_YEAR

Which is a lot haha!